<a href="https://colab.research.google.com/github/ssudhanshu488/SwinOnAlziehmer/blob/main/Increased_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import timm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score

In [12]:
# Define the dataset class
class AlzheimerDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [13]:

# Load the dataset
def load_dataset(image_folder):
    image_paths = []
    labels = []
    for image_name in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_name)
        label = image_name.split('_')[0]  # Assuming the class is the first part of the filename
        image_paths.append(image_path)
        labels.append(label)
    return image_paths, labels

In [14]:

# Preprocess the dataset (Label encoding and train/val split)
def preprocess_dataset(image_paths, labels):
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        image_paths, labels, test_size=0.2, random_state=42, stratify=labels)
    return train_paths, val_paths, train_labels, val_labels, label_encoder


In [15]:
# Define transformations
# Training transformation with augmentations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Validation transformation (only resize and normalize)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [16]:
# Load the dataset
image_folder = '/content/All_img_diff_name'
image_paths, labels = load_dataset(image_folder)
train_paths, val_paths, train_labels, val_labels, label_encoder = preprocess_dataset(image_paths, labels)

In [17]:
# Create the datasets
train_dataset = AlzheimerDataset(train_paths, train_labels, transform=train_transform)
val_dataset = AlzheimerDataset(val_paths, val_labels, transform=val_transform)


In [18]:
# Create the dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [19]:
# Load the pre-trained Swin Transformer model
model = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=3)

In [20]:
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [21]:
# Define the loss function with label smoothing (if supported)
# If your PyTorch version is older, you might remove label_smoothing parameter.
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

In [22]:
# Use AdamW optimizer and a cosine annealing scheduler
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

In [23]:
# Function to compute metrics
def compute_metrics(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    return accuracy, f1, precision

In [24]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()  # Update learning rate
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # Validation loop after each epoch
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy, f1, precision = compute_metrics(all_labels, all_preds)
    print(f'Validation Metrics after Epoch {epoch+1}:')
    print(f'Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}')


Epoch [1/10], Loss: 0.9899
Validation Metrics after Epoch 1:
Accuracy: 0.6650, F1 Score: 0.6344, Precision: 0.6955


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch [2/10], Loss: 0.8479
Validation Metrics after Epoch 2:
Accuracy: 0.5442, F1 Score: 0.5014, Precision: 0.6652


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch [3/10], Loss: 0.7788
Validation Metrics after Epoch 3:
Accuracy: 0.7597, F1 Score: 0.7544, Precision: 0.7695


...

Epoch [8/10], Loss: 0.3545
Validation Metrics after Epoch 8:
Accuracy: 0.9290, F1 Score: 0.9289, Precision: 0.9294


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch [9/10], Loss: 0.3357
Validation Metrics after Epoch 9:
Accuracy: 0.9477, F1 Score: 0.9476, Precision: 0.9478


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch [10/10], Loss: 0.3268
Validation Metrics after Epoch 10:
Accuracy: 0.9514, F1 Score: 0.9514, Precision: 0.9514


In [25]:
# Final validation after training
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [26]:
# Compute final metrics
accuracy, f1, precision = compute_metrics(all_labels, all_preds)
print('Final Validation Metrics:')
print(f'Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}')


Final Validation Metrics:
Accuracy: 0.9514, F1 Score: 0.9514, Precision: 0.9514


In [27]:
# Save the model
torch.save(model.state_dict(), 'swin_transformer_alzheimer.pth')